# 4-1 姿勢推定と OpenPose の概要

## 姿勢推定の概要
姿勢推定（pose estimation）とは1枚の画像中に含まれる複数の人物を検出し，人体の各部位も位置を同定し，それらの部位をつなぐ線（リンク）を求める技術である．
姿勢推定を行うと次のような結果が得られる．

<img src="../image/p185.png">

具体的には以下に示すように18個の部位とそれらをつなぐ37個のリンクを検出する．以下に示すリンクは一部を抜粋して表示している．

<img src="../image/p186.png">

姿勢推定によって人物の動作を推定したり姿勢の細かな分析が可能になる．

## MS COCO データセットと姿勢推定のアノテーションデータ
第4章では新たに深層学習全般に向けた汎用的なデータセットである MS COCO（Microsoft Common Objects in Context）データセットを用いる．
画像分類や物体検出，セマンティックセグメンテーション，姿勢推定などのアノテーションデータが画像とともに用意されている．
さらに，キャプションのアノテーションも用意されているため，イメージキャプショニングにデータとしても利用可能である．  
本章では特に人物が写っており，その人物の人体部位とリンクについてアノテーションが付与されている，COCO Keypoint Detection Task と呼ばれるタスクで利用されている姿勢推定用のデータを用いる．  
データに付与されているアノテーションデータは，次のような JSON ファイルで与えられる．

<img src="../image/p187.png">

画像中のメインの人物の情報（joint_self）と，その他の人物の情報（joint_others）に分けて格納されている．
そのため，同じ画像に対して異なる人物をメインとするアノテーションが存在する場合がある．　　

アノテーションデータのキー名 dataset にはデータが訓練データか検証データかによって，COCO もしくは COCO_val が格納されている．
キー名 isValidation は訓練データであれば 0.0 で，検証データであれば 1.0 が格納されている．
キー名 img_paths には画像データのリンクが格納されている．
キー名 num_keypoints には写真内のメインの人物の人体の部位がいくつアノテーションされているかが格納されており，最大で 17 になる．
検出する身体部位が 18 個でしたが MS COCO でアノテーションされているのは17 個であり，首の位置に対応するアノテーションがない．  
キー名 joint_self には首以外の 17 部位の x, y 座標とその部位の視認性情報が格納されている．
視認性情報は値が 0 のときはアノテーションの座標情報はあるが，画像内に身体部位が映っていない，値が 1 のときはアノテーションがあり画像内に身体部位も見えている，値が 2 のときは画像内に写っておらずアノテーションもない，ということを示す．
キー名 scale_provided はメインの人物を囲むバウンディングボックスの高さが 368 ピクセルに対して何倍であるかを示す．
キー名 'joint_others' には画像内の他の人物の部位情報が格納されている．
主にこれらのアノテーション情報を姿勢推定の訓練データに使用する．

## OpenPose による姿勢推定の流れ
OpenPose は CMU の Zhe Cao らによる論文 Realtime multi-person 2d pose estimation using part affinity fields が発表されてから何度か改良され，バージョンが複数あるが，ここでは原著論文をベースに実装をすすめる．  
姿勢推定は前章のセマンティックセグメンテーションと似たようなタスクである．
物体ラベルとして各部位のクラスを用意してセグメンテーションしてやれば体の部位を判定するモデルを構築できる．  
ただし，セマンティックセグメンテーションはピクセルごとのクラス分類である一方で，OpenPose の姿勢推定はピクセルごとの回帰問題である．
より具体的には，各ピクセルが18個の身体部位とその他を合わせた19個の物体である確率に対する回帰問題となる．
各身体部位について，各ピクセルの確率を回帰で求め，最も確率の高いピクセルをその身体部位の位置とする．  
姿勢推定で問題となるのは，画像内に複数の人物が写っているとき，身体部位をどうリンクさせるかという点である．
これには2通りのアプローチがある．

1. top-down 的アプローチ（single-person estimation）  
物体検出で人物を検出してから，その人物のみを切り取り姿勢推定を行う方法．
同じラベルの部位をどうリンクするかを考慮する必要がない一方で，画像中に多くの人物が写っていると処理に時間がかかったり，そもそも物体検出の精度に左右されてしまうなどの問題点がある．  
<br>
1. bottom-up 的アプローチ（multi-person estimation）  
部位間のつながり具合を表す指標として PAFs（Part Affinity Fields，下図）を導入しリンクペア問題を解決する方法．
OpenPose はこちらの方法を取っている．

<img src="../image/p190.png">

上の画像の右上と左下は推定結果であり，ピクセルごとに回帰を行い，そのピクセルが各部位である確率を求めた結果を表している．
上記の例では赤い部分の確率が最も高く，そこが左肘や左手首となる．
OpenPose では，画像内の全ての人物の身体部位の位置を，一度の推論で推定している．  
これによって，それぞれ2箇所ずつ左肘と左手首の位置が求められる．
そこで，さらに「左肘と左手首の間にあるピクセル」クラスを用意し，そのクラスに属する確率を同様にピクセルごとに回帰で求める．
この「左肘と左手首の間にあるピクセル」クラスのようなものを PAFs となる．
この PAFs を使うことで，どのペアをリンクさせるべきか自ずとわかる．
これを，すべての身体部位について行うことで，複数の人数が写っている画像でも姿勢推定が可能となる．

下の図に OpenPose による姿勢推定の流れを示す．

<img src="../image/p191.png">

### Step 1. 画像を 368×368 にリサイズ
前処理として画像のリサイズと色情報の標準化を行う．

### Step 2. 画像を OpenPose のネットワークに入力
OpenPose に前処理した画像を入力し，19×368×368 と 38×368×368 の2つの出力を得る．
それぞれのテンソルの各ピクセルは，身体部位の18クラスとそれ以外の19クラスと，37リンクの PAFs とそれ以外の38リンクである確率を表している．

### Step 3. 部位と PAFs からリンクを決定し最後に元の画像サイズに戻す
出力結果から部位ごとに座標を1点に定め，PAFs と合わせてリンクを求める．最後に画像サイズを元の大きさに戻す．